# Fans-Videos Matrix

Each row in the initial file of interactions with the user, video, and session classes is labeled based on the result of the attribute cleanup and clustering processes.

From this, the recommendations model is built.

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import scipy as sp
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from matplotlib.cm import register_cmap
from scipy import stats
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.cluster import KMeans
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import silhouette_score 
from sklearn.preprocessing import MinMaxScaler

In [29]:
df_interactions = pd.read_csv("interactions_1.csv")
df_fans = pd.read_csv("fans_final.csv")
df_videos = pd.read_csv("videos_final.csv")
#df_sessions_numerico = pd.read_csv("sessions_numerico.csv")
df_interactions

,Unnamed: 0,userId,videoId,appUser,videoDaysSinceUpload,timeZone,visitNumber,hourOfDay,totalSecondsSpent,year,...,FFA_CREATED_AT,FFA_UPDATED_AT,APP,WEB,subscriptor,timeDecay,club,matches.1,originals.1,players.1
0,23,barcafans-00a26277-3e43-4a03-960b-dbbbd494297c,2001120,1,3,1.0,34,0.0,180,2021,...,1.591154e+09,1.613725e+09,0,1,1,1.0,0.0,0.0,0.0,1.0
1,24,barcafans-00a26277-3e43-4a03-960b-dbbbd494297c,2001120,1,3,1.0,34,0.0,14,2021,...,1.591154e+09,1.613725e+09,0,1,1,1.0,0.0,0.0,0.0,1.0
2,25,barcafans-00a26277-3e43-4a03-960b-dbbbd494297c,2001120,1,3,1.0,34,0.0,18,2021,...,1.591154e+09,1.613725e+09,0,1,1,1.0,0.0,0.0,0.0,1.0
3,26,barcafans-00a26277-3e43-4a03-960b-dbbbd494297c,2001120,1,3,1.0,34,0.0,3,2021,...,1.591154e+09,1.613725e+09,0,1,1,1.0,0.0,0.0,0.0,1.0
4,27,barcafans-00a26277-3e43-4a03-960b-dbbbd494297c,2001120,1,3,1.0,34,0.0,5,2021,...,1.591154e+09,1.613725e+09,0,1,1,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64731,369316,barcafans-a8190100-7b7c-404b-8c46-664b08bfb7fd,2047460,1,0,1.0,25,23.0,351,2021,...,1.591145e+09,1.609583e+09,0,1,1,1.0,1.0,0.0,0.0,0.0
64732,369318,barcafans-a8190100-7b7c-404b-8c46-664b08bfb7fd,2045864,1,1,1.0,25,23.0,1,2021,...,1.591145e+09,1.609583e+09,0,1,1,1.0,0.0,1.0,0.0,0.0
64733,369319,barcafans-a8190100-7b7c-404b-8c46-664b08bfb7fd,2045864,1,1,1.0,25,23.0,96,2021,...,1.591145e+09,1.609583e+09,0,1,1,1.0,0.0,1.0,0.0,0.0
64734,369320,barcafans-a8190100-7b7c-404b-8c46-664b08bfb7fd,2045864,1,1,1.0,25,23.0,2,2021,...,1.591145e+09,1.609583e+09,0,1,1,1.0,0.0,1.0,0.0,0.0


In [30]:
df_interactions_cluster = pd.DataFrame()

Appending the column of user_cluster

In [32]:
df_interactions_cluster = pd.merge(df_interactions, df_fans, how='right', on='userId')
df_interactions_cluster = df_interactions_cluster.drop(['Unnamed: 0_x', 'Unnamed: 0_y'], axis=1)
df_interactions_cluster

,userId,videoId,appUser,videoDaysSinceUpload,timeZone,visitNumber,hourOfDay,totalSecondsSpent,year,month,...,FFA_UPDATED_AT,APP,WEB,subscriptor,timeDecay,club,matches.1,originals.1,players.1,user_cluster
0,barcafans-00a26277-3e43-4a03-960b-dbbbd494297c,2001120,1,3,1.0,34,0.0,180,2021,2,...,1.613725e+09,0,1,1,1.0,0.0,0.0,0.0,1.0,NaN
1,barcafans-00a26277-3e43-4a03-960b-dbbbd494297c,2001120,1,3,1.0,34,0.0,14,2021,2,...,1.613725e+09,0,1,1,1.0,0.0,0.0,0.0,1.0,NaN
2,barcafans-00a26277-3e43-4a03-960b-dbbbd494297c,2001120,1,3,1.0,34,0.0,18,2021,2,...,1.613725e+09,0,1,1,1.0,0.0,0.0,0.0,1.0,NaN
3,barcafans-00a26277-3e43-4a03-960b-dbbbd494297c,2001120,1,3,1.0,34,0.0,3,2021,2,...,1.613725e+09,0,1,1,1.0,0.0,0.0,0.0,1.0,NaN
4,barcafans-00a26277-3e43-4a03-960b-dbbbd494297c,2001120,1,3,1.0,34,0.0,5,2021,2,...,1.613725e+09,0,1,1,1.0,0.0,0.0,0.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64633,barcafans-a8190100-7b7c-404b-8c46-664b08bfb7fd,2047460,1,0,1.0,25,23.0,351,2021,2,...,1.609583e+09,0,1,1,1.0,1.0,0.0,0.0,0.0,3.0
64634,barcafans-a8190100-7b7c-404b-8c46-664b08bfb7fd,2045864,1,1,1.0,25,23.0,1,2021,2,...,1.609583e+09,0,1,1,1.0,0.0,1.0,0.0,0.0,3.0
64635,barcafans-a8190100-7b7c-404b-8c46-664b08bfb7fd,2045864,1,1,1.0,25,23.0,96,2021,2,...,1.609583e+09,0,1,1,1.0,0.0,1.0,0.0,0.0,3.0
64636,barcafans-a8190100-7b7c-404b-8c46-664b08bfb7fd,2045864,1,1,1.0,25,23.0,2,2021,2,...,1.609583e+09,0,1,1,1.0,0.0,1.0,0.0,0.0,3.0


In [23]:
nans = pd.isna(df_interactions_cluster['user_cluster'])

In [24]:
df_interactions_cluster[nans]

,userId,videoId,appUser,videoDaysSinceUpload,timeZone,visitNumber,hourOfDay,totalSecondsSpent,year,month,...,FFA_UPDATED_AT,APP,WEB,subscriptor,timeDecay,club,matches.1,originals.1,players.1,user_cluster
0,barcafans-00a26277-3e43-4a03-960b-dbbbd494297c,2001120,1,3,1.0,34,0.0,180,2021,2,...,1.613725e+09,0,1,1,1.0,0.0,0.0,0.0,1.0,NaN
1,barcafans-00a26277-3e43-4a03-960b-dbbbd494297c,2001120,1,3,1.0,34,0.0,14,2021,2,...,1.613725e+09,0,1,1,1.0,0.0,0.0,0.0,1.0,NaN
2,barcafans-00a26277-3e43-4a03-960b-dbbbd494297c,2001120,1,3,1.0,34,0.0,18,2021,2,...,1.613725e+09,0,1,1,1.0,0.0,0.0,0.0,1.0,NaN
3,barcafans-00a26277-3e43-4a03-960b-dbbbd494297c,2001120,1,3,1.0,34,0.0,3,2021,2,...,1.613725e+09,0,1,1,1.0,0.0,0.0,0.0,1.0,NaN
4,barcafans-00a26277-3e43-4a03-960b-dbbbd494297c,2001120,1,3,1.0,34,0.0,5,2021,2,...,1.613725e+09,0,1,1,1.0,0.0,0.0,0.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64628,barcafans-cc9ad96d-8a3a-4d04-ae6a-043e24bfaf6c,2043304,1,4,-5.0,28,17.0,96,2021,2,...,1.609583e+09,0,1,1,1.0,0.0,1.0,0.0,0.0,NaN
64629,barcafans-cc9ad96d-8a3a-4d04-ae6a-043e24bfaf6c,2045864,1,1,-5.0,28,17.0,19,2021,2,...,1.609583e+09,0,1,1,1.0,0.0,1.0,0.0,0.0,NaN
64630,barcafans-cc9ad96d-8a3a-4d04-ae6a-043e24bfaf6c,2045864,1,1,-5.0,28,17.0,24,2021,2,...,1.609583e+09,0,1,1,1.0,0.0,1.0,0.0,0.0,NaN
64631,barcafans-cc9ad96d-8a3a-4d04-ae6a-043e24bfaf6c,2045864,1,1,-5.0,28,17.0,2,2021,2,...,1.609583e+09,0,1,1,1.0,0.0,1.0,0.0,0.0,NaN


In the case of videos, we first remove duplicates created by different cluster results for the same video and duration.

In the DataFrame of interactions we must construct the unique identifier of videos that includes the videoId and the days since it was published.df_videos.rename (columns = {"user_cluster": "video_cluster"}) # required temporarily


In [25]:
######
df_videos = df_videos.rename(columns={"user_cluster": "video_cluster"}) # required temporarily
######
df_videos = df_videos.groupby("videoUid").filter(lambda x: len(x) == 1)
df_interactions_cluster['videoUid'] = df_interactions_cluster[['videoId', 'videoDaysSinceUpload']].astype(str).agg('_'.join, axis=1)
df_interactions_cluster = pd.merge(df_interactions_cluster, df_videos, on='videoUid')
df_interactions_cluster = df_interactions_cluster.drop(['Unnamed: 0'], axis=1)
df_interactions_cluster

,userId,videoId,appUser,videoDaysSinceUpload,timeZone,visitNumber,hourOfDay,totalSecondsSpent,year,month,...,WEB,subscriptor,timeDecay,club,matches.1,originals.1,players.1,user_cluster,videoUid,video_cluster
0,barcafans-00a26277-3e43-4a03-960b-dbbbd494297c,2001120,1,3,1.0,34,0.0,180,2021,2,...,1,1,1.0,0.0,0.0,0.0,1.0,NaN,2001120_3,1
1,barcafans-00a26277-3e43-4a03-960b-dbbbd494297c,2001120,1,3,1.0,34,0.0,14,2021,2,...,1,1,1.0,0.0,0.0,0.0,1.0,NaN,2001120_3,1
2,barcafans-00a26277-3e43-4a03-960b-dbbbd494297c,2001120,1,3,1.0,34,0.0,18,2021,2,...,1,1,1.0,0.0,0.0,0.0,1.0,NaN,2001120_3,1
3,barcafans-00a26277-3e43-4a03-960b-dbbbd494297c,2001120,1,3,1.0,34,0.0,3,2021,2,...,1,1,1.0,0.0,0.0,0.0,1.0,NaN,2001120_3,1
4,barcafans-00a26277-3e43-4a03-960b-dbbbd494297c,2001120,1,3,1.0,34,0.0,5,2021,2,...,1,1,1.0,0.0,0.0,0.0,1.0,NaN,2001120_3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50596,barcafans-3d8419cd-4b57-4369-bc42-bb7fcadc839a,1949616,1,69,1.0,15,20.0,2,2021,2,...,1,1,0.0,0.0,0.0,0.0,1.0,NaN,1949616_69,0
50597,barcafans-3d8419cd-4b57-4369-bc42-bb7fcadc839a,1949616,1,69,1.0,15,20.0,7,2021,2,...,1,1,0.0,0.0,0.0,0.0,1.0,NaN,1949616_69,0
50598,barcafans-3d8419cd-4b57-4369-bc42-bb7fcadc839a,1949616,1,69,1.0,15,20.0,8,2021,2,...,1,1,0.0,0.0,0.0,0.0,1.0,NaN,1949616_69,0
50599,barcafans-3d8419cd-4b57-4369-bc42-bb7fcadc839a,1949616,1,69,1.0,15,20.0,21,2021,2,...,1,1,0.0,0.0,0.0,0.0,1.0,NaN,1949616_69,0


In [26]:
df_interactions_cluster

,userId,videoId,appUser,videoDaysSinceUpload,timeZone,visitNumber,hourOfDay,totalSecondsSpent,year,month,...,WEB,subscriptor,timeDecay,club,matches.1,originals.1,players.1,user_cluster,videoUid,video_cluster
0,barcafans-00a26277-3e43-4a03-960b-dbbbd494297c,2001120,1,3,1.0,34,0.0,180,2021,2,...,1,1,1.0,0.0,0.0,0.0,1.0,NaN,2001120_3,1
1,barcafans-00a26277-3e43-4a03-960b-dbbbd494297c,2001120,1,3,1.0,34,0.0,14,2021,2,...,1,1,1.0,0.0,0.0,0.0,1.0,NaN,2001120_3,1
2,barcafans-00a26277-3e43-4a03-960b-dbbbd494297c,2001120,1,3,1.0,34,0.0,18,2021,2,...,1,1,1.0,0.0,0.0,0.0,1.0,NaN,2001120_3,1
3,barcafans-00a26277-3e43-4a03-960b-dbbbd494297c,2001120,1,3,1.0,34,0.0,3,2021,2,...,1,1,1.0,0.0,0.0,0.0,1.0,NaN,2001120_3,1
4,barcafans-00a26277-3e43-4a03-960b-dbbbd494297c,2001120,1,3,1.0,34,0.0,5,2021,2,...,1,1,1.0,0.0,0.0,0.0,1.0,NaN,2001120_3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50596,barcafans-3d8419cd-4b57-4369-bc42-bb7fcadc839a,1949616,1,69,1.0,15,20.0,2,2021,2,...,1,1,0.0,0.0,0.0,0.0,1.0,NaN,1949616_69,0
50597,barcafans-3d8419cd-4b57-4369-bc42-bb7fcadc839a,1949616,1,69,1.0,15,20.0,7,2021,2,...,1,1,0.0,0.0,0.0,0.0,1.0,NaN,1949616_69,0
50598,barcafans-3d8419cd-4b57-4369-bc42-bb7fcadc839a,1949616,1,69,1.0,15,20.0,8,2021,2,...,1,1,0.0,0.0,0.0,0.0,1.0,NaN,1949616_69,0
50599,barcafans-3d8419cd-4b57-4369-bc42-bb7fcadc839a,1949616,1,69,1.0,15,20.0,21,2021,2,...,1,1,0.0,0.0,0.0,0.0,1.0,NaN,1949616_69,0


In [27]:
df_interactions_cluster['target'].value_counts()

0    35517
1    15084
Name: target, dtype: int64